In [5]:
import xgboost as xgb
print('本地 xgb version : 1.0.1','运行xgb_version:', xgb.__version__)

xgb version : 1.0.1


In [6]:
import numpy as np 
import pandas as pd 
import gc
import os
from sklearn.metrics import classification_report
from scipy import stats,sparse
import time
import os
import itertools
import math
from tqdm import *
from scipy.stats import entropy, pearsonr

# import catboost as cbt
# print('catboost version :', cbt.__version__)
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score
from sklearn.metrics import roc_auc_score,recall_score,accuracy_score,log_loss,precision_score
import multiprocessing
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import label_binarize, LabelBinarizer
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.stats import pearsonr,spearmanr,kendalltau,entropy
import warnings
warnings.filterwarnings('ignore')

#### 获取数据~~~~

In [7]:
def get_data():
    train_tag = pd.read_csv('data/data32516/训练数据集_tag.csv')
    train_trd = pd.read_csv('data/data32516/训练数据集_trd.csv')
    train_beh = pd.read_csv('data/data32516/训练数据集_beh.csv')
    del train_beh['page_tm']
    train_beh.columns = ['id', 'flag', 'page_no', 'page_tm']
    print('train table: tag id {}, trd id {}, beh id {}'.format(train_tag.id.nunique(), train_trd.id.nunique(), train_beh.id.nunique()))
    print('all train id number: ', len(set(train_tag.id)|(set(train_trd.id)|set(train_beh.id))))
    # print(len(set(train_tag.id)&(set(train_trd.id)|set(train_beh.id))))
    
    print(train_tag.shape, train_trd.shape, train_beh.shape)

    test_tag = pd.read_csv('data/data32516/评分数据集_tag_b.csv')
    test_trd = pd.read_csv('data/data32516/评分数据集_trd_b.csv')
    test_beh = pd.read_csv('data/data32516/评分数据集_beh_b.csv')
    del test_beh['page_tm']
    test_beh.columns = ['id', 'page_no', 'page_tm']
    print('test table: tag id {}, trd id {}, beh id {}'.format(test_tag.id.nunique(), test_trd.id.nunique(), test_beh.id.nunique()))
    # print(len(set(test_tag.id)&(set(test_trd.id)|set(test_beh.id))))
    print('all test id number: ', len(set(test_tag.id)|(set(test_trd.id)|set(test_beh.id))))
    print(test_tag.shape, test_trd.shape, test_beh.shape)
    
    tag = train_tag.append(test_tag, sort=False).reset_index(drop=True)
    trd = train_trd.append(test_trd, sort=False).reset_index(drop=True)
    beh = train_beh.append(test_beh, sort=False).reset_index(drop=True)
    return tag, trd, beh

In [8]:
tag, trd, beh = get_data()

train table: tag id 39923, trd id 31993, beh id 11913
all train id number:  39923
(39923, 43) (1367211, 8) (934282, 4)
test table: tag id 4000, trd id 3190, beh id 1232
all test id number:  4000
(4000, 42) (142645, 7) (95669, 3)


In [13]:
for i in ['trd_id', 'beh_id']:
    tag[i] = 0
    if i == 'trd_id':
        tag.loc[tag.id.isin(trd.id.unique()), i] = 1
    else:
        tag.loc[tag.id.isin(beh.id.unique()), i] = 1
tag['sample'] = tag[['trd_id', 'beh_id','flag']].astype(str).sum(axis=1)

#### **`tag`**

In [14]:
tag_num_cols = [i for i in tag.dtypes[tag.dtypes != object].index if i not in ['id','flag','trd_id', 'beh_id','sample']]
tag_cate_cols = [i for i in tag.columns if i not in tag_num_cols+['id','flag', 'trd_id', 'beh_id','sample']]

In [17]:
cate_to_num_col = []
for col in tag_cate_cols:
    try:
        tag[col] = tag[col].replace('\\N', np.nan)
        tag[col] = tag[col].astype('float')
        # print(col)
        cate_to_num_col.append(col)
    except:
        pass

In [18]:
tag_cate_cols = [i for i in tag_cate_cols if i not in cate_to_num_col]
tag['gdr_cd'] = tag['gdr_cd'].map({'M':1,'F':0})
tag = pd.get_dummies(columns=tag_cate_cols,data=tag)

In [19]:
"""理财"""
l12_col = ['l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_gld_buy_whl_tms']
tag['l12mon_buy_sum'] = tag[l12_col].sum(axis=1)
for col in l12_col:
    tag[f'{col}_ratio'] = tag[col] / (tag['l12mon_buy_sum']+1)

In [20]:
cur_col = ['cur_debit_cnt', 'cur_credit_cnt', 'cur_debit_min_opn_dt_cnt',
'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl','hld_crd_card_grd_cd']
tag.loc[tag['cur_credit_min_opn_dt_cnt']==-1,'hld_crd_card_grd_cd'] = -1
for col in [ 'cur_debit_min_opn_dt_cnt', 'cur_credit_min_opn_dt_cnt',]:
    tag[col] = tag[col].replace(-1,0)
tag['debit-credit'] = tag['cur_debit_min_opn_dt_cnt'] - tag['cur_credit_min_opn_dt_cnt']

#### **trd**

In [23]:
def pivot_fe(data, value, ind, column, func, kk):
    pivot = pd.pivot_table(data, values=[value], index=ind, columns=column,aggfunc=func)
    pivot.columns = [value + f'_{i}_{j}_{kk}' for i in pivot.columns.levels[0] for j in pivot.columns.levels[2]]
    pivot = pivot.reset_index().fillna(0)
    return pivot

In [24]:
for col in ['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd']:
    temp = pivot_fe(trd, 'cny_trx_amt', 'id', col, ['max', 'min', 'std', 'mean', 'median', 'sum','count'], col)
    print(temp.shape)
    temp.columns = ['id'] + temp.columns.tolist()[1:]
    print(temp.shape)
    tag = tag.merge(temp, on = 'id', how='left')

(35183, 15)
(35183, 15)
(35183, 22)
(35183, 22)
(35183, 22)
(35183, 22)


In [25]:
trd['Trx_Cod2_Cd_count'] = trd['Trx_Cod2_Cd'].map(trd['Trx_Cod2_Cd'].value_counts())

In [26]:
aggs = {}
for i in ['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd']:
    # aggs[f'{i}_count'] = ['min']
    if i == 'Dat_Flg1_Cd':
        aggs[i] = ['nunique', 'count']
    else:
        aggs[i] = ['nunique']
aggs['Trx_Cod2_Cd_count'] = ['max', 'min', 'std', 'mean']
aggs['cny_trx_amt'] = ['sum', 'mean']
agg_df=trd.groupby('id').agg(aggs)
agg_df.columns = ['trd'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df=agg_df.reset_index()
tag = tag.merge(agg_df, on = 'id', how='left')

In [27]:
for i in ['cny_trx_amt_count_B_Dat_Flg1_Cd', 'cny_trx_amt_count_C_Dat_Flg1_Cd',
 'cny_trx_amt_count_A_Dat_Flg3_Cd', 'cny_trx_amt_count_B_Dat_Flg3_Cd',
 'cny_trx_amt_count_C_Dat_Flg3_Cd', 'cny_trx_amt_count_1_Trx_Cod1_Cd',
 'cny_trx_amt_count_2_Trx_Cod1_Cd', 'cny_trx_amt_count_3_Trx_Cod1_Cd']:
    tag[f'{i}_ratio'] = tag[i] / tag['trd_Dat_Flg1_Cd_count']

In [28]:
"""tfidf"""
feat = trd[['id','Trx_Cod2_Cd']]
feat['Trx_Cod2_Cd'] = feat['Trx_Cod2_Cd'].astype(str)
feat = feat.groupby(['id'])['Trx_Cod2_Cd'].agg(lambda x: ' '.join(x)).reset_index(name='Trx_Cod2_Cd_list')
tf_idf = TfidfVectorizer(max_features=30)
tf_vec = tf_idf.fit_transform(feat['Trx_Cod2_Cd_list'].values.tolist())
# svd = TruncatedSVD(n_components=16, n_iter=10, random_state=2019)
# svd_vec = svd.fit_transform(tf_vec)
tf_df = pd.DataFrame(tf_vec.toarray())
tf_df['id'] = feat['id'].values
tf_df.columns = ['Trx_Cod2_Cd_svd_vec_'+str(i+1) for i in range(30)]+ ['id']
tag = tag.merge(tf_df,on='id', how='left')
del tf_df,feat;gc.collect()

28

In [30]:
"""时间统计"""
trd['trx_tm'] = pd.to_datetime(trd['trx_tm'])
trd['trx_tm_mon'] = trd['trx_tm'].dt.month
trd['trx_tm_day'] = trd['trx_tm'].dt.day
trd['trx_tm_dayofyear'] = trd['trx_tm'].dt.dayofyear
trd['trx_tm_dayofyear'] = trd['trx_tm_dayofyear'] - trd['trx_tm_dayofyear'].min()
trd['trx_tm_hour'] = trd['trx_tm'].dt.hour
trd['trx_tm_weekday'] = trd['trx_tm'].dt.weekday
trd['trx_tm_wy'] = trd['trx_tm'].dt.weekofyear
trd['trx_tm_wy'] = trd['trx_tm_wy'] - trd['trx_tm_wy'].min()
trd['trx_tm_是否周末'] = trd['trx_tm_weekday'].apply(lambda x:1 if x >= 5 else 0)

In [31]:
aggs = {
    'trx_tm_dayofyear':['nunique', 'max', 'min', np.ptp],
}
agg_df=trd.groupby('id').agg(aggs)
agg_df.columns = ['trd'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df=agg_df.reset_index()  
tag = tag.merge(agg_df, on = 'id', how='left') 

In [32]:
#按月统计 sum
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_mon', ['sum'], 'month')
temp['5-6'] = temp['cny_trx_amt_sum_5_month'] - temp['cny_trx_amt_sum_6_month']
tag = tag.merge(temp,on='id', how='left')
#按工作日统计 mean
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_是否周末', ['mean'], '是否周末')
tag = tag.merge(temp,on='id', how='left')

In [33]:
# 按周统计 'sum','mean'
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_wy', ['sum','mean'], 'wy')
tag = tag.merge(temp,on='id', how='left')

temp = trd.groupby(['id', 'trx_tm_wy'])['cny_trx_amt'].sum().reset_index(name = 'sum')
tag['trx_tm_wy_spearmanr_wy'] = tag['id'].map(temp.groupby('id').apply(lambda x:spearmanr(x['trx_tm_wy'], x['sum'])[0]))

temp = trd.groupby(['id', 'trx_tm_wy'])['cny_trx_amt'].count().reset_index(name = 'count')
tag['trx_tm_wy_spearmanr_count'] = tag['id'].map(temp.groupby('id').apply(lambda x:spearmanr(x['trx_tm_wy'], x['count'])[0]))

In [34]:
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_wy', ['count'], 'wy')
temp.drop(columns=['cny_trx_amt_count_2_wy',
       'cny_trx_amt_count_3_wy', 'cny_trx_amt_count_4_wy',
       'cny_trx_amt_count_5_wy', 'cny_trx_amt_count_6_wy',
       'cny_trx_amt_count_7_wy'], inplace=True)
tag = tag.merge(temp,on='id', how='left')
for i in temp.columns[1:]:
    tag[i] = tag[i] / tag['trd_Dat_Flg1_Cd_count']

In [35]:
trd_b = trd[trd['Dat_Flg1_Cd'] == 'B'].copy()
for col in ['Dat_Flg3_Cd', 'Trx_Cod1_Cd']:
    temp = pivot_fe(trd_b, 'cny_trx_amt', 'id', col, ['max', 'min', 'std', 'mean'], 'b')
    print(temp.shape)
    temp.columns = ['id'] + temp.columns.tolist()[1:]
    print(temp.shape)
    tag = tag.merge(temp, on = 'id', how='left')
trd_b = trd[trd['Dat_Flg1_Cd'] == 'C'].copy()
for col in ['Dat_Flg3_Cd', 'Trx_Cod1_Cd']:
    temp = pivot_fe(trd_b, 'cny_trx_amt', 'id', col, ['max', 'min', 'std', 'mean'], 'c')
    print(temp.shape)
    temp.columns = ['id'] + temp.columns.tolist()[1:]
    print(temp.shape)
    tag = tag.merge(temp, on = 'id', how='left')

(32147, 13)
(32147, 13)
(32147, 9)
(32147, 9)
(33130, 13)
(33130, 13)
(33130, 9)
(33130, 9)


In [37]:
tag['cur_debit_cnt_div'] = tag['cur_debit_min_opn_dt_cnt'] / tag['cur_debit_cnt']
tag['cur_credit_cnt_div'] = tag['cur_credit_min_opn_dt_cnt'] / tag['cur_credit_cnt']
for sta in ['max', 'min', 'std', 'mean','median']:
    tag['cur_debit_crd_lvl_'+sta] = tag.groupby('cur_debit_crd_lvl')['cur_debit_min_opn_dt_cnt'].transform(sta)
    tag['hld_crd_card_grd_cd_'+sta] = tag.groupby('hld_crd_card_grd_cd')['cur_credit_min_opn_dt_cnt'].transform(sta)

In [40]:
for sta in ['max', 'min', 'std', 'mean','median']:
    tag['l1y_crd_card_csm_amt_dlm_cd_sum_'+sta] = tag.groupby('cur_debit_crd_lvl')['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)
    # tag['l1y_crd_card_csm_amt_dlm_cd_sum1_'+sta] = tag.groupby('l1y_crd_card_csm_amt_dlm_cd')['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)
    # tag['tot_ast_lvl_cd_mean_'+sta] = tag.groupby('hld_crd_card_grd_cd')['cny_trx_amt_sum_C_Dat_Flg1_Cd'].transform(sta)

In [41]:
temp = tag.groupby(['pl_crd_lmt_cd'])['his_lng_ovd_day'].agg(['max','min', 'std', 'mean']).reset_index()
temp.columns = ['pl_crd_lmt_cd'] + ['his_lng_ovd_day_'+i for i in ['max','min', 'std', 'mean']]
tag = tag.merge(temp,on='pl_crd_lmt_cd', how='left')
temp = tag.groupby(['pl_crd_lmt_cd'])['ovd_30d_loan_tot_cnt'].agg(['max', 'std', 'mean']).reset_index()
temp.columns = ['pl_crd_lmt_cd'] + ['ovd_30d_loan_tot_cnt_'+i for i in ['max','std', 'mean']]
tag = tag.merge(temp,on='pl_crd_lmt_cd', how='left')

In [42]:
#510
for sta in ['sum']:
    tag['pot_ast_lvl_cd_sumb_'+sta] = tag.groupby('pot_ast_lvl_cd')['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)
    tag['tot_ast_lvl_cd_sumb_'+sta] = tag.groupby('tot_ast_lvl_cd')['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)
    tag['pot_ast_lvl_cd_sumc_'+sta] = tag.groupby('pot_ast_lvl_cd')['cny_trx_amt_sum_C_Dat_Flg1_Cd'].transform(sta)
    tag['tot_ast_lvl_cd_sumc_'+sta] = tag.groupby('tot_ast_lvl_cd')['cny_trx_amt_sum_C_Dat_Flg1_Cd'].transform(sta)

    tag['hld_crd_card_grd_cd_sumb_'+sta] = tag.groupby('hld_crd_card_grd_cd')['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)
    tag['cur_debit_crd_lvl_sumb_'+sta] = tag.groupby('cur_debit_crd_lvl')['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)

tag['fagongzi'] = tag['cur_debit_min_opn_dt_cnt'] - tag['frs_agn_dt_cnt'].replace(-1,np.nan)

In [43]:
tag['beh_id'] = 1
train = tag[~tag['flag'].isnull()].reset_index(drop=True)
test = tag[tag['flag'].isnull()].reset_index(drop=True)

In [44]:
del_col=['cny_trx_amt_min_A_b', 'cny_trx_amt_min_B_b',
 'cny_trx_amt_max_1_b', 'cny_trx_amt_min_1_b', 'cny_trx_amt_min_3_b',
 'cny_trx_amt_std_1_b', 'cny_trx_amt_mean_1_b', 'cny_trx_amt_max_A_c',
 'cny_trx_amt_max_B_c', 'cny_trx_amt_max_C_c', 'cny_trx_amt_min_C_c',
 'cny_trx_amt_mean_C_c', 'cny_trx_amt_max_2_c', 'cny_trx_amt_min_2_c',
 'cny_trx_amt_std_2_c', 'cny_trx_amt_mean_2_c', 'cur_debit_crd_lvl_median',
 ]
col=[i for i in train.columns if i not in ['id', 'flag','sample']+del_col]
X_train=train[col].copy()
y_train=train['flag'].copy().astype(int)
X_test=test[col].copy()
print(X_train.shape,X_test.shape)

(39923, 266) (4000, 266)


In [2]:
K = 5
seed = 2021
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
xgb_pred_te_all = 0
xgb_auc_mean = 0
xgb_auc_mean2 = 0
f1 = []
logsocre = []
oof_xgb = np.zeros((len(X_train), ))
for i, (train_index, test_index) in enumerate(skf.split(X_train,y_train)):
    y_tr, y_val = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
    X_tr, X_val= X_train.iloc[train_index,:].copy(), X_train.iloc[test_index,:].copy()
    print( "\nFold ", i+1)

    xgb_tr = xgb.DMatrix(X_tr, y_tr)
    xgb_val = xgb.DMatrix(X_val, y_val)
    xgb_te = xgb.DMatrix(X_test)
    xgb_params = {"objective": 'binary:logistic',
                  "booster" : "gbtree",
                  "eta": 0.1,
                  "max_depth":4,#9
                  "subsample": 0.8,
                  'eval_metric':'auc',
                #   "colsample_bytree": 0.6,
                  "colsample_bylevel":0.8,
                  'gpu_id':0,
                  'tree_method':'gpu_hist',
                #   'scale_pos_weigh':np.sum(y_tr==0)/np.sum(y_tr==1),
                #   'gamma':0.2,
                  'lambda':5,                                 
                #   "thread":8,
                  "seed": 666
                  }
    watchlist = [(xgb_tr, 'train'), (xgb_val, 'eval')]
    xgb_model =xgb.train(xgb_params,
                 xgb_tr,
                 num_boost_round = 2000,
                 evals =watchlist,
                 verbose_eval=100,
                 early_stopping_rounds=100
                 ) 
    pred = xgb_model.predict(xgb_val, ntree_limit=xgb_model.best_ntree_limit)
    oof_xgb[test_index] = pred
    auc = roc_auc_score(y_val, pred)
    print( " auc_score = ", auc )
    f1.append(auc)
    print("*"*100)
    pred_te = xgb_model.predict(xgb_te,ntree_limit=xgb_model.best_ntree_limit)
    xgb_pred_te_all = xgb_pred_te_all + pred_te / K
print("="*50+'result'+"="*50)
print( " mean_auc = ", np.mean(f1) ,np.std(f1))

 mean_auc =  0.7599745786952503 0.004


In [1]:
print('本地线下分数：mean_auc=0.7599 std_auc=0.004')

本地线下分数：mean_auc=0.7599 std_auc=0.0043573498880776825


In [317]:
sub = test[['id']].copy()
sub['违约预测概率'] = xgb_pred_te_all
sub.to_csv("xgb_7599.txt", sep="\t", header=None, index=None)